In [1]:
import os
import pandas as pd
import plotly
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import subprocess
import shutil

Result load:

Make tabs as the below
Chart with all pump flows and sump water level.
If two sumps then two charts.
Add one graph with sum of all pump flows



In [22]:
model_paths = []
model_paths.append(r'J:\SEWER_AREA_MODELS\FSA\01_MASTER_MODEL\MODEL\FSA_Base.mdb')
model_paths.append(r'J:\SEWER_AREA_MODELS\VSA\01_MASTER_MODEL\MODEL\VSA_BASE_MODEL_2015.mdb')
model_paths.append(r'J:\SEWER_AREA_MODELS\NSSA\01_MASTER_MODEL\MODEL\NSSA_Base.mdb')
model_paths.append(r'J:\SEWER_AREA_MODELS\LISA\01_MASTER_MODEL\MODEL\Lisa_Base.sqlite')

In [23]:
for model_path in model_paths:
    
    print('Processing ' + os.path.basename(model_path))
    
    folder_path = os.path.dirname(model_path)

    html_path = folder_path + '\\PS_Summary'
    if not os.path.exists(html_path):
        os.makedirs(html_path)
        
    shutil.copy2('style.css', html_path + '\\style.css')
    shutil.copy2('script.js', html_path + '\\script.js')
        
        
    #Read model data
    parameter_script = r"Read_PS_Info.py"
    bat_file_path = 'Read_Parameters.bat'
    bat_file = open(bat_file_path, "w")
    bat_file.write('"' + parameter_script + '" "' + os.getcwd() + '" "' + model_path + '"')
    bat_file.close()
    result = subprocess.call([bat_file_path]) 
    if os.path.exists(model_path) == False:
        raise ValueError("The variable 'model_path' points to a path that does not exist: " + model_path)
    if result == 1: #Error
        raise ValueError("The sub process threw an error. Please Locate the bat file: " + bat_file_path + ", open it in notepad, \
        then add a new line and type in letters only: Pause. Double click the bat file to run it and it will show the error.")


    df = pd.read_csv('PS_Info.csv')
    coordinate_df = pd.read_csv('Google_Coordinates.csv',dtype={'MUID': str}).set_index('MUID')
    curves = pd.read_csv('PS_Curves.csv')
    curves = curves[(curves.HGL > -99) & (curves.HGL < 99)] #Delete fake extremes

    rtc = pd.read_csv('PS_RTC.csv')
    rtc.fillna('Otherwise',inplace=True)

    pss = list(df.PS.unique())

    f = open(html_path + '\\PS_Summary.html', "w")
    f.write('<link rel="stylesheet" href="style.css">\n')
    f.write('<script src="script.js"></script>\n')
    f.write('<link rel="stylesheet" href="style.css">\n')
    f.write('<!DOCTYPE html>\n')
    f.write('<html>\n')
    f.write('<head>\n')
    f.write('<meta charset="utf-8">\n')
    f.write('</head>\n')
    f.write('<body>\n\n')

    f.write('<div class="tab">\n')
    for i, tab in enumerate(pss):
        if i == -1: #Make tab active ----This gives wrong zoom levels so it is temporarily disabled until fixed
            f.write('  <button class="tablinks active" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
        else:
            f.write('  <button class="tablinks" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
    f.write('</div>\n')
        

    for i, ps in enumerate(pss):

    #     if ps == 'Katzie PS':
        if 1 == 1:

            if i == -1: #Make tab active ----This gives wrong zoom levels so it is temporarily disabled until fixed
                f.write('<div id="' + ps + '" class="tabcontent" style="display:block">\n')
            else:
                f.write('<div id="' + ps + '" class="tabcontent">\n')    

            df_ps = df[df.PS==ps]

            f.write('<div class="row"><div class="column">\n')  

              
            fig = go.Figure()
            
            x = 1            

            rtc_pumps = []

            for sump in df_ps.Sump.unique():

                df_ps_sump = df_ps[df_ps.Sump==sump]
                invert_level = df_ps_sump.iloc[0,2]
                ground_level = df_ps_sump.iloc[0,3]

                fig.add_shape(type="rect",x0=x, y0=invert_level, x1=x+1, y1=ground_level,line=dict(color="RoyalBlue",width=1),)
                x_mid = x + 0.5

                fig.add_annotation(x=x_mid, y=ground_level + 0.5,
                    text=sump,
                    showarrow=False,
                    arrowhead=1)

                x += 1.5

                y_axis_height = int(df_ps.Ground_Level.max())+2 - (int(df_ps.Invert_Level.min())-2)
                delta_pump_text_y = y_axis_height / 30
                pump_text_y_previous = -9999

                for index, row in df_ps_sump.iterrows():

                    pump = row['Pump']
                    if pump in list(rtc.Pump):
                        pump_rtc = rtc[rtc.Pump==pump]
                        rtc_pumps.append(pump_rtc)


                    fig.add_shape(type="line",
                        x0=x, y0=row['Start_Level'], x1=x, y1=row['Stop_Level'],
                        line=dict(
                            color="Red",
                            width=2,
                        )
                    )

                    if row['Start_Level'] > ground_level:                                
    #                     pump_text_y = ground_level + 0.5

                        if ground_level + 0.5 - pump_text_y_previous < delta_pump_text_y:  #Avoid overlapping text                  
                            pump_text_y = pump_text_y_previous + delta_pump_text_y
                        else: 
                            pump_text_y = ground_level + 0.5

                    else:
                        if row['Start_Level'] - pump_text_y_previous < delta_pump_text_y:  #Avoid overlapping text                  
                            pump_text_y = pump_text_y_previous + delta_pump_text_y
                        else: 
                            pump_text_y = row['Start_Level'] + 0.5
                    pump_text_y_previous = pump_text_y

                    fig.add_annotation(x=x, y=pump_text_y,
                        text=row['Pump'],
                        showarrow=False,
                        arrowhead=1)


                    dot_line_ys = [row['Start_Level'],row['Stop_Level']]


                    for dot_line_y in dot_line_ys:
                        fig.add_shape(type="line",
                            x0=x_mid, y0=dot_line_y, x1=x, y1=dot_line_y,
                            line=dict(
                                color="grey",
                                width=1,
                                dash="dot",
                            )
                        )  

                    x += 0.5

                x = int(x) + 1

            fig['layout']['yaxis']['range']=[int(df_ps.Invert_Level.min())-2,int(df_ps.Ground_Level.max())+2]
            fig['layout']['xaxis']['range']=[0,x]

            fig.update_layout(
#             title='PS Diagram',
            autosize=False,
            width = 500,
            height=600,
            margin=dict(
                l=50,
                r=50,
                b=0,
                t=0,
                pad=4
                ),
                xaxis_title = 'Not to scale',
                yaxis_title = 'HGL (m)'
            )
            f.write('<h1>PS Wet Well and Pump Levels</h1>\n')
            f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))  

            f.write('</div>\n')
            
            
            #********************************************************************************************************            
            f.write('<div class="column">\n')

            f.write('<h1>Pump Curves</h1>\n')
            curve_ids = set()
            for curve in list(curves.Curve.unique()):            
                if curve in list(df_ps.Pump_Curve_Max) or curve in list(df_ps.Pump_Curve_Min):
                    curve_ids.add(curve)

            if len(curve_ids) > 0:
                fig = go.Figure()
                for curve_id in curve_ids:
                    df_curve = curves[curves.Curve==curve_id]
                    fig.add_trace(go.Scatter(x=df_curve.Discharge, 
                                 y = list(df_curve.HGL), 
                                 mode='lines',name=curve_id))

                fig.update_layout(
                    showlegend=True,
                    legend=dict(
                        x=0.98,
                        xanchor="right",
                        y=0.98,
                        yanchor="top", 
                        traceorder="normal",
                        font=dict(
                            family="sans-serif",
                            size=12,
                            color="black"
                        ),
                    ),
                    autosize=False,
                    width = 500,
                    height=600,
                    margin=dict(
                        l=50,
                        r=50,
                        b=0,
                        t=0,
                        pad=4
                        ),
                    xaxis_title = 'Discharge (cms)',
                    yaxis_title = 'HGL (m)'
                )

                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn')) 
            
            f.write('</div>\n')
                
            #Create map *********************************************************************************************
            f.write('<div class="column">\n')
            f.write('<h1>Location</h1>\n')
            sump_1 = df_ps.Sump.unique()[0]
            x_coordinate = str(coordinate_df.loc[sump_1,'X'])
            y_coordinate = str(coordinate_df.loc[sump_1,'Y'])           
                               
            google_text = '<div class="mapouter"><div class="gmap_canvas">'
            google_text += '<iframe width="600" height="560" id="gmap_canvas" '
            google_text += 'src="https://maps.google.com/maps?q=' + y_coordinate + ',' + x_coordinate + '&t=&z=15&ie=UTF8&iwloc=&'
            google_text += 'output=embed&markers=' + y_coordinate + ',' + x_coordinate + '" '
            google_text += 'frameborder="1" scrolling="no" marginheight="100" marginwidth="100"></iframe>'
            google_text += '<a href="https://www.embedgooglemap.net">embedded google map</a><style>.gmap_canvas '
            google_text += '{overflow:hidden;background:none!important;height:560px;width:600px;}</style></div></div>'
            
            f.write(google_text + '\n')

            #Failed attempt to use px.scatter_mapbox. It only allows 10 per file and it needs a manual refresh to display properly
#             data = {
#                 'PS': sump_1,
#                 'X': x_coordinate,
#                 'Y': y_coordinate
#             }
#             df_map = pd.DataFrame(data, index=[0])            
            
#             fig_map = px.scatter_mapbox(df_map,
#                                     lon=df_map.X,
#                                     lat=df_map.Y,
#                                     zoom=15,
#                                     width=600,
#                                     height=600,
#                                     title='Location of ' + ps)

#             fig_map.update_layout(mapbox_style='open-street-map')
#             fig_map.update_layout(margin={'r':20,'t':30,'l':10,'b':30})
            
#             f.write('<div id="map">\n')
#             f.write(fig_map.to_html(full_html=False, include_plotlyjs='cdn')) 
#             f.write('</div>\n')
                    
            f.write('</div></div>\n')

            #********************************************************************************************************  

            
        
            #Pump info tables****************************************************************************************
            f.write('<h3>Pump Information</h3>\n')

            df_pumps = df_ps[df_ps.columns[4:]].copy()

            df_pumps['Control'] = 'No'
            for index, row in df_pumps.iterrows():
                if row['Pump'] in list(rtc.Pump):
                    df_pumps.loc[index,'Control'] = 'Yes'

            f.write('\n<table>\n')
            f.write('  <tr>\n')

            for column in df_pumps.columns:
                f.write('    <th>' + column.replace('_',' ') + '</th>\n')

            f.write('  </tr>\n')

            for index, row in df_pumps.iterrows():
                f.write('  <tr>\n')
                for column in df_pumps.columns:
                    if isinstance(row[column],str):
                        f.write('    <td>' + str(row[column]) + '</th>\n')
                    else:
                        if np.isnan(row[column]):
                            f.write('    <td></th>\n')
                        else:
                            f.write('    <td>' + str(row[column]) + '</th>\n')
                f.write('  </tr>\n')                   
            f.write('</table>\n')

            if len(rtc_pumps) > 0:
                for rtc_pump in rtc_pumps:
                    pump = rtc_pump.iloc[0,0]
                    rtc_pump = rtc_pump[rtc_pump.columns[2:]]
                    f.write('<h4>Control for Pump ' + pump + '</h4>\n')

                    f.write('\n<table>\n')
                    f.write('  <tr>\n')

                    for column in rtc_pump.columns:
                        f.write('    <th>' + column.replace('_',' ') + '</th>\n')
                    f.write('  </tr>\n')
                    for index, row in rtc_pump.iterrows():
                        f.write('  <tr>\n')
                        for column in rtc_pump.columns:
                            f.write('    <td>' + str(row[column]) + '</th>\n')
                        f.write('  </tr>\n')                   
                    f.write('</table>\n')

            f.write('</div>\n')

    f.write('</body>\n')
    f.write('</html>\n')
    f.close()
    
print("Finished")




Processing FSA_Base.mdb
Processing VSA_BASE_MODEL_2015.mdb
Processing NSSA_Base.mdb
Processing Lisa_Base.sqlite
Finished
